### **Imports**

In [1]:
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
print("✅ Demo tools loaded.")

✅ Demo tools loaded.


### **Model**

In [2]:
# Initialize the open-source embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

print(f"✅ Model '{model_name}' is loaded and ready.")

/var/folders/m7/xxvzrlmd0wddkflz97l7nq7r0000gn/T/ipykernel_11152/813670556.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


✅ Model 'sentence-transformers/all-MiniLM-L6-v2' is loaded and ready.


### **Text-to-Vector**
*(Main component of RAG)*

Here is a plain English question being converted to numeric vector.

**"How many accidents in Virginia involved alcohol?"**

In [3]:
# Here is a plain English question
question = "How many accidents in Virginia involved alcohol?"

# Let's run it through the model
vector = embeddings.embed_query(question)

print("--- Your Question ---")
print(f"'{question}'")

print("\n--- Becomes a 'Vector' (a list of numbers) ---")
#print("\nFirst 5 numbers of the vector:")
#print(np.array(vector)[:5])
print(np.array(vector))
print(f"\nTotal length of the vector: {len(vector)}")

--- Your Question ---
'How many accidents in Virginia involved alcohol?'

--- Becomes a 'Vector' (a list of numbers) ---
[ 8.25532675e-02  7.39611080e-03  1.54931024e-02  2.28705741e-02
  3.94746102e-02  1.02309123e-01 -2.83401608e-02  1.93100106e-02
 -6.40363097e-02  3.13794166e-02  7.02017099e-02 -3.06326542e-02
 -3.37345451e-02  3.43769714e-02 -4.58711274e-02 -1.94535460e-02
  7.81070860e-03 -4.56623957e-02 -2.20975243e-02  1.99681204e-02
 -1.71534829e-02  2.57411618e-02  6.69329613e-02 -1.41509240e-02
 -6.80635357e-03  2.49181110e-02 -5.12567023e-03  7.08811209e-02
  1.24479868e-02 -4.45041135e-02  1.95077322e-02  1.39297610e-02
  1.87728778e-02  8.95657169e-04 -1.95257701e-02 -1.02194510e-01
  5.49994037e-02  5.42069264e-02  4.48841155e-02  1.21720489e-02
  1.54754017e-02 -3.69361527e-02  2.77480520e-02  1.40515668e-02
  1.14831431e-02 -1.33910319e-02 -5.47143258e-02  1.45685999e-02
  3.93598080e-02  1.14458902e-02  5.54015972e-02  3.19396779e-02
  1.83511768e-02  7.18316622e-03  

 **Now, let's create two other sentences to compare**

In [ ]:
text_similar = "What is the number of drunk driving crashes in VA?"
text_dissimilar = "What's the weather like in California?"


vector_similar = embeddings.embed_query(text_similar)
vector_dissimilar = embeddings.embed_query(text_dissimilar)
print("✅ Embedded two more sentences to compare.")

In [ ]:
# We need to reshape the vectors for the function
v_question = np.array(vector).reshape(1, -1)
v_similar = np.array(vector_similar).reshape(1, -1)
v_dissimilar = np.array(vector_dissimilar).reshape(1, -1)

# Now, let's use 'cosine similarity' to see how "close" they are.
# A score of 1.0 is a perfect match.
sim_similar = cosine_similarity(v_question, v_similar)[0][0]
sim_dissimilar = cosine_similarity(v_question, v_dissimilar)[0][0]

print("--- Similarity Results (1.0 = identical) ---")
print(f"Similarity to 'drunk driving': {sim_similar:.4f}")
print(f"Similarity to 'California weather': {sim_dissimilar:.4f}")

### **Retreive Data from Your SQL Database**

In [ ]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("✅ RAG tools loaded. We will build this from scratch.")

### **Load Your FARS Data**

Loading small, fast sample from your CSV to act as our "database."

In [ ]:
# Load a sample of your CSV data
df = pd.read_csv("/Users/rafaelviray/Documents/FARS/Datasets & Ingestion Scripts/accident_master.csv", nrows=50)

print("✅ Loaded sample of FARS 'accident_master.csv'.")
print("Here's a preview:")
print(df[['ST_CASE', 'YEAR', 'STATE', 'FATALS']])

### **Building the "R" (Retrieval) Component**

This is the core **"technical content"**. We serialize and index the data.

In [ ]:
# 1. Serialize: Convert rows into simple "documents"
print("Serializing CSV rows into text snippets...")
documents = []
for _, row in df.iterrows():
    # Create a simple text snippet for each accident
    content = (
        f"Accident case {row['ST_CASE']} took place in {row['STATENAME']} "
        f"(state code {row['STATE']}) in the year {row['YEAR']}. "
        f"This incident resulted in {row['FATALS']} fatalities."
    )
    doc = Document(page_content=content, metadata={"st_case": row['ST_CASE']})
    documents.append(doc)

# 2. Index: Load the embedding model and build the vector store
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("Building live vector store... (This is the 'R' in RAG)")
vectorstore = Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

print("✅ Vector store is live.")

In [ ]:
# Initialize our open-source LLM
llm = ChatOllama(model="llama3")

# This is a specific question from our CSV data
question = "How many fatalities were in accident case 47157?"

print(f"--- Asking the 'Dumb' LLM ---")
print(f"QUESTION: {question}\n")

# We invoke the LLM *without* RAG. It will fail.
response = llm.invoke(question)

print("--- RESPONSE (Hallucination) ---")
print(response)

In [ ]:
# 1. Augment: We create a prompt template
# It will be "augmented" with the context we retrieve
template = """
Answer the question based ONLY on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 2. Generate: We build the full chain
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

# This chain links all our steps together
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Full RAG Chain is built.")
print("This chain will now RETRIEVE, AUGMENT, and GENERATE.")

In [ ]:
print(f"--- Asking the 'Smart' RAG Chain ---")
print(f"QUESTION: {question}\n")

# Invoke the full RAG chain
response = rag_chain.invoke(question)

print("--- RESPONSE (Grounded in FARS) ---")
print(response)